In [ ]:
#pip install plotly==5.6.0

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.templates
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import pandas as pd
import datetime
from datetime import date, timedelta
import re

In [ ]:
def cut(x):
    return x[:-3]

In [ ]:
def format_date1(x):
    return datetime.datetime.strptime(x, "%Y-%m-%d %X").strftime("%Y-%m-%d")

In [ ]:
# New Subscribers Over Time

#TODO Why not purchased_at inplace of subscriber_since?

# *** NEW SUBSCRIBERS OVER TIME *** 

# format new subscriptions data (using only subscriber_since data and grouping by date)
all_data_df = pd.read_csv("subscriptions.csv")
all_data_df = all_data_df.rename(columns={'subscriber_since': 'Date Purchased'})
new_subs_df = pd.DataFrame(all_data_df['Date Purchased'])
new_subs_df = new_subs_df.dropna(how = 'all')
new_subs_df["Date Purchased"] = new_subs_df["Date Purchased"].apply(cut)
new_subs_df["Date Purchased"] = new_subs_df["Date Purchased"].apply(format_date1)
num_new_subs = new_subs_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscribers').reset_index()
# Add a new column, 'Number of Subscribers From Trialers'. Initialize entire column to 0 
num_new_subs['Number of Subscribers (Trialers)'] = 0
print(num_new_subs.head(), "\n")

# **** NEW TRIALERS OVER TIME ***

# format new subscriptions from trialers data (using only subscriber since_data and was_trialer and grouping by date)
new_subs_from_trial_df = all_data_df[['Date Purchased', 'was_trial']].copy()
new_subs_from_trial_df = new_subs_from_trial_df.dropna()
print(new_subs_from_trial_df.head(), "\n\n", "Trials over time", "\n")
new_subs_from_trial_df = new_subs_from_trial_df[new_subs_from_trial_df.was_trial != "f"]
print(new_subs_from_trial_df.head(), "\n\n", "Frequency Table: Trialers Over Time", "\n")
# Delete the 'was_trial' column, currently holding trues-only, and build a "trues-only" frequency
# table. 
del new_subs_from_trial_df["was_trial"]
new_subs_from_trial_df["Date Purchased"] = new_subs_from_trial_df["Date Purchased"].apply(cut)
new_subs_from_trial_df["Date Purchased"] = new_subs_from_trial_df["Date Purchased"].apply(format_date1)
num_new_subs_from_trial = new_subs_from_trial_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscribers (Trialers)').reset_index()
print(num_new_subs_from_trial.head(), "\n", "Total Subscriptions and Trials Component Over Time", "\n")

# *** COMBINED: NEW SUBSCRIBERS AND TRIALERS OVER TIME ***

# combine two dfs
all_subs = pd.merge(num_new_subs, num_new_subs_from_trial, on="Date Purchased", how="outer")
# There'll two columns with the lede, "Number of Subscribers (Trialers)..."
del all_subs["Number of Subscribers (Trialers)_x"]
all_subs = all_subs.rename(columns={'Number of Subscribers (Trialers)_y': 'Number of Subscribers (Trialers)'})
all_subs['Number of Subscribers (Trialers)'].fillna(0, inplace=True)

#  plot data
df = all_subs
template = "plotly_white"
fig = px.line(df, x="Date Purchased", y=df.columns,
              hover_data={"Date Purchased": "|%B %d, %Y"},
              template=template,
              title='Subscribers Over Time, Including Trials View')

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

#fig.show()
fig.write_html('index.html', auto_open=True)
